<a href="https://colab.research.google.com/github/hazhyni/07-crypto-policy-x-review/blob/main/Crypto_Sentiment_Analysis_Pre_Post_Policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crypto Sentiment Analysis

##Import Library

In [ ]:
!pip install snscrape pandas scikit-learn nltk sastrawi
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

##Node Package Installation


In [ ]:
# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update

!sudo apt-get install nodejs -y

!node -v

## Pre-Policy

###Crawl Data Pra(2009-2018)

In [ ]:
filename = "sentimen_kripto_2009_2018.csv"
search_keyword = "kripto since:2009-01-01 until:2018-12-31 lang:in"
limit = 10000

!npx --yes tweet-harvest@latest -o "{filename}" -s "{search_keyword}" -l "{limit}" --token "f59f3fb97889932c58beba249dfab6b07eb81809"

###Preprocessing App Review Pre-Policy

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path_pra = f"tweets-data/{filename}"


# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path_pra, delimiter=",")

df.rename(columns={"full_text": "tweet"}, inplace=True)
selected_columns = df[["created_at","tweet", "lang"]]


# Display the DataFrame
display(df.head(3))

In [ ]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

In [ ]:
# Download necessary NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# Download the punkt_tab resource required for word_tokenize/sent_tokenize
nltk.download('punkt_tab')

# Load the dataset (assuming df is already loaded as in the preceding code)
# df = pd.read_csv('sentimen_kripto_2009_2019.csv', delimiter=',')

# --- Cleaning ---
def clean_text(text):
    text = text.lower() # lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove urls
    text = re.sub(r'\@\w+', '', text) # remove mentions
    text = re.sub(r'\#\w+', '', text) # remove hashtags
    text = re.sub(r'\d+', '', text) # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    text = text.strip() # remove leading/trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip() # remove multiple spaces
    return text

df['cleaned_tweet'] = df['tweet'].apply(clean_text)

# --- Case Folding (already done in clean_text) ---

# --- Tokenization ---
df['tokenized_tweet'] = df['cleaned_tweet'].apply(lambda x: nltk.word_tokenize(x))

# --- Stopword Removal ---
list_stopwords = set(stopwords.words('indonesian'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

df['stopwords_removed_tweet'] = df['tokenized_tweet'].apply(remove_stopwords)

# --- Stemming ---
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_words(tokens):
    return [stemmer.stem(word) for word in tokens]

# Stemming after stopword removal
df['stemmed_tweet'] = df['stopwords_removed_tweet'].apply(stem_words)

# Display the progress (optional)
print("\nDataFrame after preprocessing:")
display(df[['tweet', 'cleaned_tweet', 'tokenized_tweet', 'stopwords_removed_tweet', 'stemmed_tweet']].head(5))

###Data Labelling

In [ ]:
# Manual Labelling Function with Sensitivity
def manual_labeling_sensitive(text):

    # Define keywords and phrases for each sentiment category (adjust these based on your specific data)
    positive_keywords = [
        "naik", "untung", "profit", "bagus", "optimis", "sukses", "menguat", "bullish",
        "peluang", "potensi", "menarik", "rekomendasi beli", "layak investasi", "akan naik",
        "prospek cerah", "baik", "aman", "terpercaya", "menguntungkan","keren","positif","beruntung","bangun","baik","layak","tinggi","dipercaya"
    ]
    negative_keywords = [
        "turun", "rugi", "loss", "buruk", "pesimis", "gagal", "melemah", "bearish",
        "resiko", "bahaya", "rugi besar", "jual", "akan turun", "prospek suram", "jelek",
        "tidak aman", "penipuan", "merugikan", "hati-hati","jelek","hacker","retas","menurun","negatif","volatilitas","rendah"
    ]
    neutral_keywords = [
        "harga", "pasar", "kripto", "bitcoin", "ethereum", "transaksi",
        "berita", "analisis", "prediksi", "pergerakan", "update", "informasi", "data",
        "volume", "kapitalisasi pasar", "blockchain", "wallet", "exchange"
    ]

    # Simple check for negative sentiment first
    for keyword in negative_keywords:
        if keyword in text:
            return 'Negative'

    # Check for positive sentiment
    for keyword in positive_keywords:
        if keyword in text:
            return 'Positive'

    # If neither strong positive nor negative keywords are found, check for neutral
    # This part is more sensitive to data differences. We check if any neutral keywords are present
    # without significant positive or negative indicators.
    for keyword in neutral_keywords:
        if keyword in text:
            # You could add more nuanced checks here if needed,
            # e.g., checking for negation before positive/negative words,
            # but for basic sensitivity, just the presence of neutral words
            # after checking for strong sentiment can suffice for some cases.
            return 'Neutral'

    # If no matching keywords are found, label as Neutral by default
    return 'Neutral'

# Apply the manual labeling function to the processed text column
# Choose the column that is most suitable for labeling after preprocessing.
# 'stemmed_tweet' is usually a good candidate as it's been cleaned and stemmed.
if 'stemmed_tweet' in df.columns:
    df['label_result'] = df['stemmed_tweet'].apply(manual_labeling_sensitive)
    print("Hasil Labelling dengan Lexicon Based Sentiment Analysis.")
    display(df[['tweet', 'stemmed_tweet', 'label_result']].head())
else:
    print("Column 'stemmed_tweet' not found for manual labeling. Please check your preprocessing steps.")

# You can now analyze the distribution of labels
if 'label_result' in df.columns:
    sentiment_counts = df['label_result'].value_counts()
    print("\nSentiment Label Distribution:")
sentiment_counts

# You can also save the labeled data to a new CSV file
# df.to_csv('dataset_kripto_labeled_manual.csv', index=False)
# print("\nLabeled data saved to 'dataset_kripto_labeled_manual.csv'")


###Output

In [ ]:
# prompt: buatkan hasil visualisasi wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all the stemmed tweets into a single string
all_words = ' '.join([text for text in df['stemmed_tweet'].astype(str)])

# Create the WordCloud object
wordcloud = WordCloud(width = 800, height = 400,
                background_color ='white',
                stopwords = list_stopwords, # Use the stopwords list from preprocessing
                min_font_size = 10).generate(all_words)

# Display the generated image:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()


In [ ]:
# prompt: buatkan analisis k-fold cross validationnya

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Prepare data for K-Fold Cross-Validation
# We will use the 'stemmed_tweet' as the feature and 'label_result' as the target
X = df['stemmed_tweet'].astype(str)  # Convert to string to handle potential non-string entries
y = df['label_result']

# Initialize KFold
n_splits = 5  # You can change the number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Initialize the classifier (using Support Vector Machine as an example)
model = SVC(kernel='linear')

# Lists to store evaluation results
accuracy_scores = []
classification_reports = []

print(f"Performing {n_splits}-Fold Cross-Validation...")

# Perform K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"\n--- Fold {fold + 1}/{n_splits} ---")

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and transform the training data
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

    # Transform the testing data
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # Train the model
    model.fit(X_train_tfidf, y_train)

    # Predict on the test data
    y_pred = model.predict(X_test_tfidf)

    # Evaluate the model
    fold_accuracy = accuracy_score(y_test, y_pred)
    fold_report = classification_report(y_test, y_pred, zero_division=0) # zero_division=0 handles cases where a class has no predicted samples

    accuracy_scores.append(fold_accuracy)
    classification_reports.append(fold_report)

    print(f"Fold {fold + 1} Accuracy: {fold_accuracy:.4f}")
    print(f"Fold {fold + 1} Classification Report:\n{fold_report}")

# Print overall results
print("\n--- Overall K-Fold Cross-Validation Results ---")
print(f"Average Accuracy across {n_splits} folds: {sum(accuracy_scores)/n_splits:.4f}")

# You can also print the report for each fold or the report from the last fold
# print("\nClassification Report from the last fold:")
# print(classification_reports[-1])

# To get an aggregate report across all folds, you would typically combine predictions
# or average metrics in a more complex way, but the average accuracy gives a good overall idea.


In [ ]:
# Create a pie chart in grayscale
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', colors=['#808080', '#A9A9A9', '#D3D3D3'], startangle=140)
plt.title('Hasil Analisis Sentimen', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()


##Pasca-Policy

###Crawl Data

In [ ]:
filename = "sentimen_kripto_2019_2025.csv"
search_keyword = "kripto since:2019-01-01 until:2025-05-31 lang:in,id"
limit = 10000

!npx --yes tweet-harvest@latest -o "{filename}" -s "{search_keyword}" -l "{limit}" --token "f59f3fb97889932c58beba249dfab6b07eb81809"

###Preprocessing

In [ ]:
# Specify the path to your CSV file
file_path_pra = f"tweets-data/{filename}"


# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path_pra, delimiter=",")

df.rename(columns={"full_text": "tweet"}, inplace=True)
selected_columns = df[["created_at","tweet", "lang"]]


# Display the DataFrame
display(df.head(3))

In [ ]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

In [ ]:
# Download necessary NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# Download the punkt_tab resource required for word_tokenize/sent_tokenize
nltk.download('punkt_tab')

# Load the dataset (assuming df is already loaded as in the preceding code)
# df = pd.read_csv('sentimen_kripto_2009_2019.csv', delimiter=',')

# --- Cleaning ---
def clean_text(text):
    text = text.lower() # lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove urls
    text = re.sub(r'\@\w+', '', text) # remove mentions
    text = re.sub(r'\#\w+', '', text) # remove hashtags
    text = re.sub(r'\d+', '', text) # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    text = text.strip() # remove leading/trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip() # remove multiple spaces
    return text

df['cleaned_tweet'] = df['tweet'].apply(clean_text)

# --- Case Folding (already done in clean_text) ---

# --- Tokenization ---
df['tokenized_tweet'] = df['cleaned_tweet'].apply(lambda x: nltk.word_tokenize(x))

# --- Stopword Removal ---
list_stopwords = set(stopwords.words('indonesian'))

def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

df['stopwords_removed_tweet'] = df['tokenized_tweet'].apply(remove_stopwords)

# --- Stemming ---
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_words(tokens):
    return [stemmer.stem(word) for word in tokens]

# Stemming after stopword removal
df['stemmed_tweet'] = df['stopwords_removed_tweet'].apply(stem_words)

# Display the progress (optional)
print("\nDataFrame after preprocessing:")
display(df[['tweet', 'cleaned_tweet', 'tokenized_tweet', 'stopwords_removed_tweet', 'stemmed_tweet']].head(5))

###Sentiment Classification

In [ ]:
# Manual Labelling Function with Sensitivity
def manual_labeling_sensitive(text):

    # Define keywords and phrases for each sentiment category (adjust these based on your specific data)
    positive_keywords = [
        "naik", "untung", "profit", "bagus", "optimis", "sukses", "menguat", "bullish",
        "peluang", "potensi", "menarik", "rekomendasi beli", "layak investasi", "akan naik",
        "prospek cerah", "baik", "aman", "terpercaya", "menguntungkan","keren","positif","beruntung","bangun","baik","layak","tinggi","dipercaya"
    ]
    negative_keywords = [
        "turun", "rugi", "loss", "buruk", "pesimis", "gagal", "melemah", "bearish",
        "resiko", "bahaya", "rugi besar", "jual", "akan turun", "prospek suram", "jelek",
        "tidak aman", "penipuan", "merugikan", "hati-hati","jelek","hacker","retas","menurun","negatif","volatilitas","rendah"
    ]
    neutral_keywords = [
        "harga", "pasar", "kripto", "bitcoin", "ethereum", "transaksi",
        "berita", "analisis", "prediksi", "pergerakan", "update", "informasi", "data",
        "volume", "kapitalisasi pasar", "blockchain", "wallet", "exchange"
    ]

    # Simple check for negative sentiment first
    for keyword in negative_keywords:
        if keyword in text:
            return 'Negative'

    # Check for positive sentiment
    for keyword in positive_keywords:
        if keyword in text:
            return 'Positive'

    # If neither strong positive nor negative keywords are found, check for neutral
    # This part is more sensitive to data differences. We check if any neutral keywords are present
    # without significant positive or negative indicators.
    for keyword in neutral_keywords:
        if keyword in text:
            # You could add more nuanced checks here if needed,
            # e.g., checking for negation before positive/negative words,
            # but for basic sensitivity, just the presence of neutral words
            # after checking for strong sentiment can suffice for some cases.
            return 'Neutral'

    # If no matching keywords are found, label as Neutral by default
    return 'Neutral'

# Apply the manual labeling function to the processed text column
# Choose the column that is most suitable for labeling after preprocessing.
# 'stemmed_tweet' is usually a good candidate as it's been cleaned and stemmed.
if 'stemmed_tweet' in df.columns:
    df['label_result'] = df['stemmed_tweet'].apply(manual_labeling_sensitive)
    print("Hasil Labelling dengan Lexicon Based Sentiment Analysis.")
    display(df[['tweet', 'stemmed_tweet', 'label_result']].head())
else:
    print("Column 'stemmed_tweet' not found for manual labeling. Please check your preprocessing steps.")

# You can now analyze the distribution of labels
if 'label_result' in df.columns:
    sentiment_counts = df['label_result'].value_counts()
    print("\nSentiment Label Distribution:")
sentiment_counts

# You can also save the labeled data to a new CSV file
# df.to_csv('dataset_kripto_labeled_manual.csv', index=False)
# print("\nLabeled data saved to 'dataset_kripto_labeled_manual.csv'")


###Output

In [ ]:
# prompt: buatkan hasil visualisasi wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Join all the stemmed tweets into a single string
all_words = ' '.join([text for text in df['stemmed_tweet'].astype(str)])

# Create the WordCloud object
wordcloud = WordCloud(width = 800, height = 400,
                background_color ='white',
                stopwords = list_stopwords, # Use the stopwords list from preprocessing
                min_font_size = 10).generate(all_words)

# Display the generated image:
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()


In [ ]:
# Create a pie chart in grayscale
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', colors=['#808080', '#A9A9A9', '#D3D3D3'], startangle=140)
plt.title('Hasil Analisis Sentimen', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()
